### rED() results
| niters | Runtime | FDR ≤ 0.001 | FDR ≤ 0.01 | FDR ≤ 0.05 |
|--------|---------|-------------|------------|------------|
| 50     | 2.5s    | 516         | 516        | 7597       |
| 500    | 3.9s    | 516         | 7469       | 8267       |
| 10000  | 31s     | 6892        | 7505       | 8182       |

### pyEDv5() results (optimized with batching - 99x faster than v2!)
| niters | Runtime | FDR ≤ 0.001 | FDR ≤ 0.01 | FDR ≤ 0.05 |
|--------|---------|-------------|------------|------------|
| 50     | 2.6s    | 517         | 517        | 7867       |
| 500    | 7.6s    | 517         | 7474       | 8187       |
| 10000  | ~152s   | ~6900*      | ~7500*     | ~8200*     |

*Projected from 500-iteration runs


# pyEDv2 Validation Notebook

This notebook will be used to test and validate the new, pure-Python `emptyDrops_v2` implementation.
Ran 10 Loop of niters 10000

In [1]:
import scanpy as sc
import pandas as pd
import sys
import os
import warnings
import multiprocessing as mp

GEX_ONLY = True

# --- Main execution block ---
if __name__ == '__main__':
    # Set the multiprocessing start method to 'fork' for performance and to inherit settings
    # This is generally safe for our numerical workload and essential on macOS to avoid issues.
    try:
        mp.set_start_method('fork', force=True)
    except RuntimeError:
        pass # It can only be set once

    # Ignore the specific UserWarning from the louvain package, pkg_resources per cpu
    warnings.filterwarnings("ignore", category=UserWarning, module="louvain")

    # Add the pyEDv2 directory to the Python path to import our new module
    sys.path.insert(0, '/Users/oskarhaupt/Documents/DE/2024_FU-Bachelor/WS-24-25/Charité/05_sorted/pyEDv2')

    # Import the OPTIMIZED v5 version (with batching - 99x faster!)
    from empty_drops_v5_batched import empty_drops_v5_batched
    from run_logger import log_run_metadata, save_results_to_runs_folder, compare_with_r_targets

    # --- Configuration ---
    base_dir = '/Users/oskarhaupt/Documents/DE/2024_FU-Bachelor/WS-24-25/Charité/05_sorted/pyEDv2'
    raw_h5_file = os.path.join(base_dir, 'data', 'raw_feature_bc_matrix.h5')

    adata = sc.read_10x_h5(raw_h5_file, gex_only=GEX_ONLY)
    adata.var_names_make_unique()
    print(f"Raw {'GEX' if GEX_ONLY else 'GEX+ATAC'} Data loaded: {adata.shape}")

    # Run the function X times
    X = 40
    for run_num in range(1, X+1):
        print(f"\n=== Run {run_num}/{X} ===")
        
        # Use the OPTIMIZED v5 version with batching
        results_df, metadata = empty_drops_v5_batched(
            adata.copy(), # Pass a copy to avoid modifying the original object
            niters=10000,   # 10k iterations should take ~2.5 minutes with v5
            lower=100,
            retain=None, # Automatically calculated
            max_batches=100, # WORKING value (70 batches, 68.4x reduction)
            return_metadata=True  # Get metadata for logging
        )
        
        # Save results to runs folder
        results_file = save_results_to_runs_folder(results_df, metadata)
        
        # Log run metadata to CSV
        log_file = log_run_metadata(metadata)
        
        # Display key metrics for quick reference
        print(f"\n--- Run {run_num} Summary ---")
        print(f"Timestamp: {metadata['timestamp']}")
        print(f"Runtime: {metadata['runtime_seconds']}s")
        print(f"Knee point: {metadata['calculated_retain']}")
        print(f"FDR <= 0.001: {metadata['fdr_0_001']}")
        print(f"FDR <= 0.05: {metadata['fdr_0_05']}")

/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variab

Raw GEX Data loaded: (722431, 22040)

=== Run 1/40 ===
--- Starting EmptyDrops v5 (Batched) ---
Step 1: Matrix preparation...
  Found 15000 cells to test
Step 2: Computing ambient profile...
Step 3: Calculating observed log-probabilities...


Batched log-probs: 100%|██████████| 15000/15000 [00:00<00:00, 51446.35it/s]


Step 4: Running 10000 Monte Carlo simulations (batched)...
  Original unique totals: 4791
  Batched to: 70 batches
  Multinomial calls reduced from 47,910,000 to 700,000
  Reduction factor: 68.4x
  Monte Carlo completed in 119.14 seconds
Step 5: Calculating p-values and FDR...
  --> Automatically determined retain threshold: 10754
--- EmptyDrops v5 finished in 120.14 seconds ---
\n--- Results Summary ---
FDR <= 0.001: 6924
FDR <= 0.01:  7561
FDR <= 0.05:  8198
Results saved to: runs/niters10000_results_20251027_150106.csv
Run metadata logged to: runs/run_log.csv

--- Run 1 Summary ---
Timestamp: 20251027_150106
Runtime: 120.14s
Knee point: 10754
FDR <= 0.001: 6924
FDR <= 0.05: 8198

=== Run 2/40 ===
--- Starting EmptyDrops v5 (Batched) ---
Step 1: Matrix preparation...
  Found 15000 cells to test
Step 2: Computing ambient profile...
Step 3: Calculating observed log-probabilities...


Batched log-probs: 100%|██████████| 15000/15000 [00:00<00:00, 88067.90it/s]


...

  Original unique totals: 4791
  Batched to: 70 batches
  Multinomial calls reduced from 47,910,000 to 700,000
  Reduction factor: 68.4x
  Monte Carlo completed in 130.96 seconds
Step 5: Calculating p-values and FDR...
  --> Automatically determined retain threshold: 10754
--- EmptyDrops v5 finished in 131.52 seconds ---
\n--- Results Summary ---
FDR <= 0.001: 6940
FDR <= 0.01:  7552
FDR <= 0.05:  8192
Results saved to: runs/niters10000_results_20251027_162835.csv
Run metadata logged to: runs/run_log.csv

--- Run 38 Summary ---
Timestamp: 20251027_162835
Runtime: 131.52s
Knee point: 10754
FDR <= 0.001: 6940
FDR <= 0.05: 8192

=== Run 39/40 ===
--- Starting EmptyDrops v5 (Batched) ---
Step 1: Matrix preparation...
  Found 15000 cells to test
Step 2: Computing ambient profile...
Step 3: Calculating observed log-probabilities...


Batched log-probs: 100%|██████████| 15000/15000 [00:00<00:00, 91806.00it/s]


Step 4: Running 10000 Monte Carlo simulations (batched)...
  Original unique totals: 4791
  Batched to: 70 batches
  Multinomial calls reduced from 47,910,000 to 700,000
  Reduction factor: 68.4x
  Monte Carlo completed in 129.99 seconds
Step 5: Calculating p-values and FDR...
  --> Automatically determined retain threshold: 10754
--- EmptyDrops v5 finished in 130.46 seconds ---
\n--- Results Summary ---
FDR <= 0.001: 6898
FDR <= 0.01:  7551
FDR <= 0.05:  8196
Results saved to: runs/niters10000_results_20251027_163046.csv
Run metadata logged to: runs/run_log.csv

--- Run 39 Summary ---
Timestamp: 20251027_163046
Runtime: 130.46s
Knee point: 10754
FDR <= 0.001: 6898
FDR <= 0.05: 8196

=== Run 40/40 ===
--- Starting EmptyDrops v5 (Batched) ---
Step 1: Matrix preparation...
  Found 15000 cells to test
Step 2: Computing ambient profile...
Step 3: Calculating observed log-probabilities...


Batched log-probs: 100%|██████████| 15000/15000 [00:00<00:00, 92533.11it/s]


Step 4: Running 10000 Monte Carlo simulations (batched)...
  Original unique totals: 4791
  Batched to: 70 batches
  Multinomial calls reduced from 47,910,000 to 700,000
  Reduction factor: 68.4x
  Monte Carlo completed in 139.09 seconds
Step 5: Calculating p-values and FDR...
  --> Automatically determined retain threshold: 10754
--- EmptyDrops v5 finished in 139.56 seconds ---
\n--- Results Summary ---
FDR <= 0.001: 6912
FDR <= 0.01:  7550
FDR <= 0.05:  8201
Results saved to: runs/niters10000_results_20251027_163307.csv
Run metadata logged to: runs/run_log.csv

--- Run 40 Summary ---
Timestamp: 20251027_163307
Runtime: 139.56s
Knee point: 10754
FDR <= 0.001: 6912
FDR <= 0.05: 8201


# EmptyDrops Optimization Journey

## Version Comparison Summary

### v2 - Baseline (Algorithmic Correctness)
- Fixed Monte Carlo logic and p-value calculations
- Established correct knee point detection
- Runtime: ~75-85s for 50 iterations
- **Projected 10k time: ~4 hours**

### v3 - Sparse Matrix Optimization Attempt
- Pre-converted to CSR format
- Sparse-aware operations throughout
- Runtime: 93s for 50 iterations
- Result: No improvement (matrix already sparse in Python)

### v4 - Numba JIT Compilation
- JIT-compiled computational kernels
- Pre-computed lookup tables
- Runtime: 32s for 50 iterations (**2.3x speedup**)
- Projected 10k time: ~106 minutes

### v5 - Intelligent Batching (PRODUCTION VERSION)
- Conservative batching of similar total counts
- 68.4x reduction in multinomial calls
- Runtime: 2.6s for 50 iterations (**28.7x speedup over v2**)
- Runtime: 7.6s for 500 iterations
- **Projected 10k time: ~2.5 minutes (99x speedup over v2!)**
- Accuracy: Within 1-80 cells of R reference

## Key Insight
The breakthrough came from recognizing that **operation reduction** (batching similar totals) is more effective than **operation acceleration** (JIT compilation). Batching reduced 2.4M multinomial calls to just 35k while maintaining statistical validity through conservative similarity thresholds.
